In [47]:
import pandas as pd
import json
#from datetime import timedelta
#import calendar
import numpy as np
#import re
#from sklearn.preprocessing import MinMaxScaler

# Load in dataset from assignment 2

In [22]:
# read in tsv dataset
reports = pd.read_csv('../../Data/reports_v3.tsv', sep='\t')

print(reports)

# get an overview of all columsn
# for col in reports.columns:
    # print(col)


        Report Type     Id    Class                Submitted Date  \
0            Report  13038  Class A  Saturday, November 12, 2005.   
1            Report   8792  Class B       Thursday, June 3, 2004.   
2            Report   1255  Class B     Tuesday, October 5, 1999.   
3     Media Article    658      NaN   Wednesday, January 23, 2013   
4            Report  11616  Class B          Friday, May 6, 2005.   
...             ...    ...      ...                           ...   
5462         Report   1411  Class A    Tuesday, January 23, 2001.   
5463  Media Article     59      NaN      Wednesday, June 18, 1980   
5464         Report  14714  Class A         Friday, May 19, 2006.   
5465         Report  12884  Class A    Tuesday, October 25, 2005.   
5466         Report    798  Class B        Monday, June 26, 2000.   

                                               Headline  Year  Season  \
0     Snowmobiler has encounter in deep snow near Po...  2004  Winter   
1     Four nocturnal hike

# Selecting columns of interest and preparing for visualization

In [57]:
# define a list of only the columns I need for my visualization
columns_of_interest = ['National Park Visitation Count', 'BigFoot Hotspot', 'Year']

# keep only the columns of interest in the datase
df = reports[columns_of_interest]

#print(df)

# inspect value counts and nan values
#print(df['Year'].value_counts())
#print(df['BigFoot Hotspot'].value_counts())
#print(df['National Park Visitation Count'].value_counts())
#print(df['National Park Visitation Count'].isna().sum())
#print(df['BigFoot Hotspot'].isna().sum())
#print(df['Year'].isna().sum())

# dropping nan rows (these only exist for national park visiation count)
df.dropna(inplace=True)

# dropping rows where the year is 0
df = df[df['Year'] != 0]

# converting the national park visitation count to integer (currently float)
df['National Park Visitation Count'] = df['National Park Visitation Count'].astype(int)

print(df)

# print(df['Year'].value_counts())

# Group the data by 'Year' and 'BigFoot Hotspot' and compute the median visitation count
median_visitation_counts = df.groupby(['Year', 'BigFoot Hotspot'])['National Park Visitation Count'].median().reset_index()

# Pivot the data to have years as rows and columns for false and true hotspots
pivot_table = median_visitation_counts.pivot(index='Year', columns='BigFoot Hotspot', values='National Park Visitation Count')

# Drop rows with missing values
pivot_table.dropna(inplace=True)

# Rename the columns for clarity
pivot_table.columns = ['Non_Hotspots_Median', 'Hotspots_Median']

# Create a MinMaxScaler instance
#scaler = MinMaxScaler()

#visitation_counts_1 = pivot_table['Non_Hotspots_Median'].values.reshape(-1, 1)
#visitation_counts_1 = pivot_table['Hotspots_Median'].values.reshape(-1, 1)

# Fit and transform the visitation counts using MinMaxScaler
#pivot_table['Non_Hotspots_Median'] = scaler.fit_transform(visitation_counts_1)
#pivot_table['Hotspots_Median'] = scaler.fit_transform(visitation_counts_1)

# adding year column in (currently the index)
pivot_table.reset_index(inplace=True)

print(pivot_table)

      National Park Visitation Count  BigFoot Hotspot  Year
0                             244232            False  2004
1                             243719            False  2003
2                               5550            False  1998
4                              56787            False  2004
5                              28331            False  2000
...                              ...              ...   ...
5461                         2590624            False  2000
5462                         2540699            False  1994
5464                         2463442            False  2005
5465                         1360898            False  1984
5466                         2838233            False  2000

[4200 rows x 3 columns]
    Year  Non_Hotspots_Median  Hotspots_Median
0   1979            1118800.0         963726.0
1   1980            1160588.0        1268256.0
2   1981            1286316.0        1233671.0
3   1982            1015580.0        1007300.0
4   1983            

C:\Users\tberg_cl8rsaz\AppData\Local\Temp\ipykernel_14288\99739560.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(inplace=True)


# Convert to json

In [58]:
# Convert DataFrame to a dictionary of records and wrap in another dictionary
dic = {'BFRO': pivot_table.to_dict(orient='records')}

# Write the dictionary to a JSON file with pretty printing
with open('../../Dataset1/BFRO_vis_1.json', 'w') as f:
    json.dump(dic, f, indent=2)
